In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd
import numpy as np

In [2]:
# Load the Trained model, scalar pickel, onehot

model = load_model('classification_model.keras')

with open('label_encoder_gender.pkl', 'rb') as f :
    gender_encoder = pickle.load(f)

with open('onehot_encoder_geo.pkl', 'rb') as f :
    geo_encoder = pickle.load(f)

with open('scaler.pkl', 'rb') as f :
    scaler = pickle.load(f)

c:\Users\vaibh\anaconda3\envs\ann-env\lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
# Example input data

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
# One-hot encode 'Geography'

geo_encoded = geo_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geo_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\vaibh\anaconda3\envs\ann-env\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
print(type(input_data))

input_df = pd.DataFrame([input_data])
input_df


<class 'dict'>


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
input_df['Gender'] = gender_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [7]:
input_df = input_df.drop('Geography', axis=1)
input_df = pd.concat([input_df, geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
# Scaling the Input Data DF

input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.5462356 ,  0.92295821,  0.09061905, -0.70539966, -0.25290648,
         0.79579729,  0.65153774,  0.99700449, -0.84008335,  1.00300451,
        -0.58119931, -0.57581067]])

In [9]:
prediction = model.predict(input_scaled)

print(prediction)
if prediction[0][0] > 0.5 :
    print(f'Will Churn, Model Prediction = {prediction[0][0]}')
else : 
    print(f'Will not Churn, Model Prediction = {prediction[0][0]*100:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
[[0.04448105]]
Will not Churn, Model Prediction = 4.45


In [10]:
# [[0.06037656]]
# Will not Churn, Model Prediction = 6.04